In [6]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
newdf = pd.read_csv('data/order_full_information.csv')

# Check the first few rows
print(newdf.shape)
print(newdf.head())

(50, 7)
   Unnamed: 0 order_created_date  order_id product_name  quantity category  \
0           0       '2024-12-01'         1   Product_11         1  Dresses   
1           1       '2024-12-01'         2   Product_17         2   Shirts   
2           2       '2024-12-01'         3   Product_19         1   Shirts   
3           3       '2024-12-01'         4   Product_12         1  Jackets   
4           4       '2024-12-01'         5   Product_11         2  Dresses   

   price  
0  69.06  
1  98.51  
2  51.94  
3  50.99  
4  69.06  


In [9]:
import requests
import os

API_KEY = os.getenv("currencyapikey")
url = f"https://api.freecurrencyapi.com/v1/latest?apikey={API_KEY}&currencies=USD&base_currency=BRL"

response = requests.get(url)
data = response.json()

# Extract exchange rate
brl_to_usd = data["data"]["USD"]

# Step 3: Calculate total_price_us and total_price_br
newdf["total_price_br"] = newdf["price"] * newdf["quantity"]
newdf["total_price_us"] = newdf["total_price_br"] * brl_to_usd

# Step 4: Select required columns
result_df = newdf[[
    "order_created_date", "order_id", "product_name", "quantity",
    "category","total_price_br", "total_price_us"
]]

In [10]:
print(result_df.shape)
print(result_df.head())

(50, 7)
  order_created_date  order_id product_name  quantity category  \
0       '2024-12-01'         1   Product_11         1  Dresses   
1       '2024-12-01'         2   Product_17         2   Shirts   
2       '2024-12-01'         3   Product_19         1   Shirts   
3       '2024-12-01'         4   Product_12         1  Jackets   
4       '2024-12-01'         5   Product_11         2  Dresses   

   total_price_br  total_price_us  
0           69.06       12.256088  
1          197.02       34.965166  
2           51.94        9.217799  
3           50.99        9.049202  
4          138.12       24.512175  


In [11]:
# 1. Date with max number of orders
orders_per_date = result_df.groupby("order_created_date")["order_id"].nunique()
print(orders_per_date)
max_orders_date = orders_per_date.idxmax()
print(max_orders_date)
max_orders_count = orders_per_date.max()
print(max_orders_count)

order_created_date
'2024-12-01'     8
'2024-12-02'     5
'2024-12-03'     3
'2024-12-05'     6
'2024-12-06'    10
'2024-12-07'     4
'2024-12-09'     7
'2024-12-10'     7
Name: order_id, dtype: int64
'2024-12-06'
10


In [12]:
# 2. Most demanded product and total sell price
product_qty = result_df.groupby("product_name")["quantity"].sum()
print(product_qty)
most_demanded_product = product_qty.idxmax()
print(most_demanded_product)
most_demanded_qty = product_qty.max()
print(most_demanded_qty)
total_sell_price = result_df[result_df["product_name"] == most_demanded_product]["total_price_us"].sum()
print(total_sell_price)

product_name
Product_1      8
Product_10     1
Product_11     4
Product_12     4
Product_13    13
Product_14    10
Product_15     5
Product_17    10
Product_18    12
Product_19    19
Product_2      2
Product_20    12
Product_3      2
Product_4      7
Product_5     20
Product_6      3
Product_7     11
Product_8      3
Name: quantity, dtype: int64
Product_5
20
158.26786862250003


In [16]:
# 3. Top 3 most demanded categories
top_categories = result_df.groupby("category")["quantity"].sum().nlargest(3)
print(top_categories)

category
Shirts     50
Jackets    30
Pants      29
Name: quantity, dtype: int64


In [19]:
rows = [
    {
        "kpi": "Max orders date",
        "value": max_orders_date,
        "extra": max_orders_count
    },
    {
        "kpi": "Most demanded product",
        "value": most_demanded_product,
        "extra": f"{most_demanded_qty} units, ${total_sell_price:.2f} USD"
    },
    {
        "kpi": "Top 3 most demanded categories",
        "value": top_categories,
        "extra": f"{top_categories} units, ${total_sell_price:.2f} USD"
    }
]

kpi_df = pd.DataFrame(rows)
kpi_df.to_csv("data/kpi_product_orders.csv", index=False)